# ZF automated proof checking

It turns out that proofs in "raw ZF" are exponentially long.

To get moving with any speed we immediately need to start proving "meta theorems" which provide
"derived rules" beyond those directly given at the machine code level.

For example, the machine code just has the substitution rule

```
INTERNAL SUBSTITUTION RULE.
¬a ∨ a
====================
⊤
```

where `a` is an atom, i.e., a statement of the form `x ∈ y`.

But we immediately need

```
EXTERNALLY DERIVED METATHEOREM. Let φ be any formula (maybe with free variables). Then the following substitution can be realized by a sequence of low-level substitutions.
¬φ ∨ φ
====================
⊤
```
which we can prove by induction on the formula length.

In fact, to keep exponentially picking up speed (proving more interesting theorems), we need to keep reasoning at higher levels still. There still will always exist a base-level proof, but it might be like $10^{100}$ steps long.

In [1]:
import string
import matplotlib.pyplot as plt

In [2]:
# We could have a machine language with only NAND.
# But this will cause an exponential blowup in the term length.
# To not get an exponential blowup, we need at least NOT, AND, IMPLIES, and IFF.
# (Or something equivalent, e.g. we could have XOR instead of AND.)
# On the other hand, compiling ∃ to ¬∀¬ does not cause any blowup.

TOKENS = [
    ('LPAREN', '('),
    ('RPAREN', ')'),
    ('NOT', r'\neg'),
    ('AND', r'\wedge'),
    ('IMPLIES', r'\implies'),
    ('IFF', r'\iff'),
    ('FORALL', r'\forall'),
    ('CONTAINS', r'\in'),
    ('TRUE', '\top'),
    ####################
    ('OR', r'\vee'),
    ('UNION', r'\cup'),
    ('INTERSECTION', r'\cap'),
    ('EXISTS', r'\exists'),
    ('SUBSET', r'\subset'),
    ('EQUALS', '='),
    ###################
    ('FORALL', '∀'),
    ('EXISTS', '∃'),
    ('NOT', '¬'),
    ('CONTAINS', '∈'),
    ('AND', '∧'),
    ('OR', '∨'),
    ('IMPLIES', '⇒'),
    ('IFF', '⇔'),
    ('SUBSET', '⊂'),
    ('TRUE', '⊤'),
#     ('NOT', r'\not'),
]

def tokenize(formula):
    formula = '(%s)' % formula
    def helper():
        i = 0
        while i < len(formula):
            if formula[i] == ' ':
                i += 1
                continue
            for t, v in TOKENS:
                if formula[i:i+len(v)] == v:
                    yield (t, None)
                    i += len(v)
                    break
            else:
                assert formula[i] in string.ascii_lowercase, (i, formula[i])
                yield ('VARIABLE', formula[i])
                i += 1
    return list(helper())

In [3]:
BINOPS = ['OR', 'AND', 'IMPLIES', 'CONTAINS', 'SUBSET', 'EQUALS', 'IFF']
QUANTS = ['FORALL', 'EXISTS']
PRINT_SYMS = {
    'OR': '∨',
    'AND': '∧',
    'IMPLIES': '⇒',
    'IFF': '⇔',
    'FORALL': '∀',
    'EXISTS': '∃',
    'CONTAINS': '∈',
    'SUBSET': '⊂',
    'NOT': '¬',
    'EQUALS': '=',
    'TRUE': '⊤',
}

class ASTNode:
    def __init__(self, node_type, node_value=None):
        self.t = node_type
        self.v = node_value
        self.meta = dict(free_vars=set(), quant_vars=set())
        
    def copy_from(self, node):
        self.t = node.t
        self.v = node.v
        self.meta = node.meta
    
    def __repr__(self):
        if isinstance(self.v, list):
            if self.t in BINOPS:
                return '(%s %s %s)' % (self.v[0], PRINT_SYMS[self.t], self.v[1])
            elif self.t in QUANTS:
                return '%s%s%s' % (PRINT_SYMS[self.t], self.v[0], self.v[1])
            elif self.t == 'NOT':
                return '%s%s' % (PRINT_SYMS[self.t], self.v[0])
            elif self.t == 'TRUE':
                return PRINT_SYMS[self.t]
            else:
                return '%s(%s)' % (self.t, ' '.join(str(n) for n in self.v))
        else:
            return str(self.v if self.t == 'VARIABLE' else self.t)

def parse(tokens):
    def parse_expression(index):
        if tokens[index][0] == 'LPAREN':
            index += 1
            node = ASTNode('GROUP', [])
            while tokens[index][0] != 'RPAREN':
                child_node, index = parse_expression(index)
                node.v.append(child_node)
            return node, index + 1 # skip RPAREN
        else:
            return ASTNode(*tokens[index]), index+1

    ast_root, _ = parse_expression(0)
    return ast_root

In [4]:
# infers the proper grouping for not and forall/exists
def group_terms(node):
    def helper(node):
        if not isinstance(node.v, list):
            return node
        children = node.v = [helper(n) for n in node.v]
        
        def group_at(i): # tries to form a group at index i
            if i >= len(children)-1:
                return
            
            if children[i].t == 'NOT':
                op_node = children.pop(i)
                group_at(i)
                a_node = children.pop(i)
                assert a_node.t == 'GROUP', a_node
                children.insert(i, ASTNode('GROUP', [op_node, a_node]))
                
            if children[i].t in ['FORALL', 'EXISTS']:
                op_node = children.pop(i)
                x_node = children.pop(i)
                assert x_node.t == 'VARIABLE'
                group_at(i)
                phi_node = children.pop(i)
                assert phi_node.t == 'GROUP'
                children.insert(i, ASTNode('GROUP', [op_node, x_node, phi_node]))
                
        i = 0
        while i < len(children):
            group_at(i)
            i += 1
        
        if len(children) == 1:
            return children[0] # eliminate unary groups
        else:
            node.v = children
            return node
        
    return helper(node)

In [5]:
def transform_noop(node):
    if not isinstance(node.v, list):
        return node
    children = node.v = [transform_noop(n) for n in node.v]
    return node
    
def extract_quantifiers(node):
    if not isinstance(node.v, list):
        return node
    children = node.v = [extract_quantifiers(n) for n in node.v]
    if node.t == 'GROUP':
        if children[0].t in ['FORALL', 'EXISTS'] and not children[0].v:
            assert children[1].t == 'VARIABLE', children
            assert len(children) == 3
            node.t = children[0].t
            node.v = [children[1], children[2]]
    return node

def extract_binops(node):
    def helper(node):
        if not isinstance(node.v, list):
            return node
        children = node.v = [helper(n) for n in node.v]
        if node.t == 'GROUP':
            if children[1].t in BINOPS and not children[1].v:
                assert len(children) == 3
                node.t = children[1].t
                node.v = [children[0], children[2]]
        return node
    return helper(node)

def extract_not(node):
    def helper(node):
        if not isinstance(node.v, list):
            return node
        children = node.v = [helper(n) for n in node.v]
        if node.t == 'GROUP':
            if children[0].t == 'NOT' and not children[0].v:
                assert len(children) == 2, children
                node.t = children[0].t
                node.v = [children[1]]
        return node
    return helper(node)

In [6]:
# check the formula for basic validity: e.g., (x ∈ (y ∈ x)) is invalid
def tag_logic(node):
    def helper(node):
        if isinstance(node.v, list):
            for n in node.v:
                helper(n)
        if node.t == 'VARIABLE':
            node.meta['is_logic'] = False
        elif node.t in ['NOT', 'AND', 'OR', 'IMPLIES', 'IFF']:
            assert all(n.meta['is_logic'] for n in node.v)
            node.meta['is_logic'] = True
        elif node.t in ['EXISTS', 'FORALL']:
            assert node.v[0].t == 'VARIABLE' # this is already known anyway
            assert node.v[1].meta['is_logic']
            node.meta['is_logic'] = True
#         elif node.t in ['UNION', 'INTERSECTION']:
#             assert all(not n.meta['is_logic'] for n in node.v)
#             node.meta['is_logic'] = False
        elif node.t in ['CONTAINS', 'SUBSET', 'EQUALS']:
            assert all(n.t == 'VARIABLE' for n in node.v)
            node.meta['is_logic'] = True
        else:
            assert False
    helper(node)
    assert node.meta['is_logic']

In [7]:
# # optional zero-level rules which cause an exponential blowup in formula length, and the duplication of
# # quantified terms.
# def compile_level0(tree):
#     def helper(node):
#         if not isinstance(node.v, list):
#             return node
#         node.v = [helper(n) for n in node.v]

#         if node.t == 'IMPLIES':
#             a, b = node.v
#             node = ASTNode('NOT', [ASTNode('AND', [a, ASTNode('NOT', [b])])])
            
#         if node.t == 'IFF':
#             a, b = node.v
#             not1 = ASTNode('NOT', [ASTNode('AND', [a, ASTNode('NOT', [b])])])
#             not2 = ASTNode('NOT', [ASTNode('AND', [b, ASTNode('NOT', [a])])])
#             node = ASTNode('AND', [not1, not2])
            
#         return node
#     return helper(tree)

# first-level compilation rules
def compile_level1(tree):
    def helper(node):
        if not isinstance(node.v, list):
            return node
        node.v = [helper(n) for n in node.v]
        
        if node.t == 'OR':
            a, b = node.v
            node = ASTNode('NOT', [ASTNode('AND', [ASTNode('NOT', [a]), ASTNode('NOT', [b])])])

        if node.t == 'EXISTS':
            x, phi = node.v
            node = ASTNode('NOT', [ASTNode('FORALL', [x, ASTNode('NOT', [phi])])])
            
        return node
    return helper(tree)

# second level compilation rules which depend on the first level
def compile_level2(tree):
    
    var_index = [0]
    def get_var():
        var_index[0] += 1
        return 'c%d' % var_index[0]
    
    def helper(node):
        if not isinstance(node.v, list):
            return node
        node.v = [helper(n) for n in node.v]

        if node.t == 'SUBSET':
            a, b = node.v
            v = get_var()
            imp = ASTNode('IMPLIES', [ASTNode('CONTAINS', [ASTNode('VARIABLE', v), a]),
                                      ASTNode('CONTAINS', [ASTNode('VARIABLE', v), b])])
            node = ASTNode('FORALL', [ASTNode('VARIABLE', v), imp])

        # compilation-time axiom of extensionality
        if node.t == 'EQUALS':
            a, b = node.v
            v = get_var()
            iff = ASTNode('IFF', [ASTNode('CONTAINS', [ASTNode('VARIABLE', v), a]),
                                  ASTNode('CONTAINS', [ASTNode('VARIABLE', v), b])])
            node = ASTNode('FORALL', [ASTNode('VARIABLE', v), iff])
            
        return node
    return helper(tree)

In [8]:
def tag_quantification(node):
    """
    check the formula for quantification validity: e.g., the following are invalid
    - ∀x∀x(x ∈ x) ; something already quantified is being quantified again
    - (x ∈ y) ; everything needs to be quantified
    - ∀x(x ∈ x) ∧ ∀x(x ∈ x) ; quantified variables should be unique.
    """
    def helper(node):
        if isinstance(node.v, list):
            for n in node.v:
                helper(n)
        if node.t == 'VARIABLE':
            node.meta['free_vars'] = set([node.v])
            node.meta['quant_vars'] = []
            
        elif node.t in ['NOT', 'AND', 'OR', 'IMPLIES', 'IFF', 'CONTAINS']: # 'CONTAINS', 'SUBSET', 'EQUALS'
            # free vars bubbling from below can of course not be unique
            node.meta['free_vars'] = set(v for n in node.v for v in n.meta['free_vars'])
            # quantified variables bubbling from below must be unique
            q_vars = [v for n in node.v for v in n.meta['quant_vars']]
            assert len(q_vars) == len(set(q_vars))
            node.meta['quant_vars'] = set(q_vars)
            
        elif node.t in ['EXISTS', 'FORALL']:
            assert node.v[0].t == 'VARIABLE'
            q_var = node.v[0].v
            subtree_free = [v for n in node.v for v in n.meta['free_vars']]
            subtree_quantified = [v for n in node.v for v in n.meta['quant_vars']]
#             assert q_var in subtree_free # commented out because it's OK to quantify things that aren't free.
            assert q_var not in subtree_quantified
            node.meta['free_vars'] = set([v for v in subtree_free if v != q_var])
            node.meta['quant_vars'] = set([q_var] + subtree_quantified)
            
        else:
            assert False, node
    helper(node)
    assert not node.meta['free_vars']

In [9]:
def compile_tree(formula):
    tokens = tokenize(formula)
    
    tree = parse(tokens)
    
    tree = group_terms(tree)
    tree = extract_quantifiers(tree)
    tree = extract_binops(tree)
    tree = extract_not(tree)
    
    tag_logic(tree)
    tree = compile_level2(tree)
    tree = compile_level1(tree)
    tag_quantification(tree)
    return tree

In [10]:
# Axiom of the empty set
# formula = r'\exists x (\forall y (\not (y \in x)))'
# formula = r'∃x∀y¬(y ∈ x)'

# Theorem: every set contains the empty set
# formula = r'\forall x ((\forall y (\neg (y \in x))) \implies (\forall a (x \subset a)))'
# formula = r'∀x(∀y¬(y ∈ x) ⇒ ∀a(x ⊂ a))'

# no two-cycles
# formula = r'∀x∀y \not ((x \in y) \wedge (y \in x))'

# Axiom of powerset
# formula = r'∀x∃y∀z((z ⊂ x) ⇒ (z ∈ y))'

# formula = '∀x∀y((x = y) \implies (x ⊂ y))'

# A provable tautology.
# formula = '∀x∀y((x = y) \iff ((x ⊂ y) \wedge (y ⊂ x)))'
# formula = '∀x∀y((x = y) ⇔ ((x ⊂ y) ∧ (y ⊂ x)))'

# de Morgan's law. A provable tautology.
formula = '∀x∀a∀b∀c(((x ∈ a) ∧ ((x ∈ b) ∨ (x ∈ c))) ⇒ (((x ∈ a) ∧ (x ∈ b)) ∨ ((x ∈ a) ∧ (x ∈ c))))'

# formula = r'(((x \in a) \vee (x \in b)) \wedge \neg (x \in a)) \implies (x \in b)'
# formula = r'((x \in a) \wedge \neg (x \in a))'

print(formula)
tree = compile_tree(formula)
print(tree.meta['quant_vars'])

∀x∀a∀b∀c(((x ∈ a) ∧ ((x ∈ b) ∨ (x ∈ c))) ⇒ (((x ∈ a) ∧ (x ∈ b)) ∨ ((x ∈ a) ∧ (x ∈ c))))
{'a', 'b', 'x', 'c'}


In [25]:
def demorgan_forward(tree):
    """
    φ ∧ ¬(¬Ψ ∧ ¬ξ)
    ====================
    ¬(¬(φ ∧ Ψ) ∧ ¬(φ ∧ ξ))
    """
    n = tree
    assert n.t == 'AND'
    phi = n.v[0]
    assert n.v[1].t == 'NOT'
    assert n.v[1].v[0].t == 'AND'
    assert n.v[1].v[0].v[0].t == 'NOT'
    psi = n.v[1].v[0].v[0].v[0]
    assert n.v[1].v[0].v[1].t == 'NOT'
    xi = n.v[1].v[0].v[1].v[0]
    new_tree = ASTNode('NOT', [ASTNode('AND', [
        ASTNode('NOT', [ASTNode('AND', [phi, psi])]),
        ASTNode('NOT', [ASTNode('AND', [phi, xi])]),
    ])])
    tree.copy_from(new_tree)
    
def self_remove_tautology(tree):
    """
    φ ⇒ φ
    ====================
    ⊤
    """
    n = tree
    assert n.t == 'IMPLIES'
    assert str(n.v[0]) == str(n.v[1])
    new_tree = ASTNode('TRUE', [])
    tree.copy_from(new_tree)
    
def elim_quant(tree):
    """
    ∀xφ
    ====================
    φ
    """
    n = tree
    assert n.t == 'FORALL'
    q_var = n.v[0].t
    assert q_var not in n.v[1].meta['free_vars']
    new_tree = n.v[1]
    tree.copy_from(new_tree)

def apply_substitution(tree, rule):
    """
    Recursively apply a substitution rule, bubbling eliminated quantified variables up the tree.
    """
    if tree.t == 'VARIABLE':
        return
    
    for n in tree.v:
        apply_substitution(n, rule)
    try:
        rule(tree)
    except:
        pass
    
    # Bubble up eliminated free and quantified variables
    tree.meta['free_vars'] = set(v for n in tree.v for v in n.meta['free_vars'])
    tree.meta['quant_vars'] = set(v for n in tree.v for v in n.meta['quant_vars'])
    if tree.t in ['FORALL', 'EXISTS']:
        tree.meta['quant_vars'].add(tree.v[1].t)

In [19]:
formula = '∀x∀a∀b∀c(((x ∈ a) ∧ ((x ∈ b) ∨ (x ∈ c))) ⇒ (((x ∈ a) ∧ (x ∈ b)) ∨ ((x ∈ a) ∧ (x ∈ c))))'
tree = compile_tree(formula)
print(tree)

∀x∀a∀b∀c(((x ∈ a) ∧ ¬(¬(x ∈ b) ∧ ¬(x ∈ c))) ⇒ ¬(¬((x ∈ a) ∧ (x ∈ b)) ∧ ¬((x ∈ a) ∧ (x ∈ c))))


In [20]:
apply_substitution(tree, demorgan_forward)
tree

∀x∀a∀b∀c(¬(¬((x ∈ a) ∧ (x ∈ b)) ∧ ¬((x ∈ a) ∧ (x ∈ c))) ⇒ ¬(¬((x ∈ a) ∧ (x ∈ b)) ∧ ¬((x ∈ a) ∧ (x ∈ c))))

In [21]:
apply_substitution(tree, self_imp_forward)
tree

∀x∀a∀b∀c⊤

In [22]:
apply_substitution(tree, elim_quant)
tree

⊤

In [ ]:
"""
We would like a system of deduction rules which generate theorems.
Where all theorems are of the form "[subset of ZF] => [Theorem]".
We cannot include all of ZF in the left hand side because of the (infinite) specification schema.


THE FOLLOWING SUBSTITUTION RULES ARE VALID TO APPLY TO ANY FRAGMENT OF A FORMULA.
THE LETTERS x AND y DENOTE ARBITRARY VARIABLES.
GREEK SYMBOLS DENOTE FRAGMENTS OF FORMULAS IN WHICH AT LEAST THE EXPLICITLY
PASSED VARIABLES ARE FREE. THEY MAY CONTAIN OTHER FREE VARIABLES AS WELL.
IF BOTH x AND y APPEAR IN THE RULE, AND A FORMULA DOES NOT EXPLICITLY
HAVE x OR y PASSED, THEN THAT VARIABLE IS NEITHER FREE NOR QUANTIFIED IN THE FORMULA.

FOR A SUBSTITUTION RULE:

LOCAL SUBTITUTION.
α
====================
β

IT BECOMES TWO TRANSFORMATION RULES:

GLOBAL TRANSFORMATION.
XαY
\/\/\/\/\/\/\/\/\/\/
XβY

GLOBAL TRANSFORMATION.
XβY
\/\/\/\/\/\/\/\/\/\/
XαY

WHERE X AND Y ARE ANY SUBSTRINGS SUCH THAT XαY IS A VALID FORMULA.

################################################################################


I. Properties of ∧, ¬.

SUBSTITUTION. [double negation]
φ
====================
¬¬φ

SUBSTITUTION. [identity]
φ
====================
φ ∧ φ

SUBSTITUTION. [commutativity]
φ ∧ Ψ
====================
Ψ ∧ φ

SUBSTITUTION. [associativity]
φ ∧ (Ψ ∧ ξ)
====================
(φ ∧ Ψ) ∧ ξ

SUBSTITUTION. [distributivity]
φ ∧ ¬(¬Ψ ∧ ¬ξ)
====================
¬(¬(φ ∧ Ψ) ∧ ¬(φ ∧ ξ))

SUBSTITUTION. [excluded middle 1]
φ
====================
φ ∧ ¬(Ψ ∧ ¬Ψ)

TRANSFORMATION.
φ ∧ Ψ
\/\/\/\/\/\/\/\/\/\/
φ


II. Properties of ∀.

SUBSTITUTION.
∀x∀yφ(x, y)
====================
∀y∀xφ(x, y)

TRANSFORMATION.
∀x∀yφ(x, y)
\/\/\/\/\/\/\/\/\/\/
∀xφ(x, x)

SUBSTITUTION. [φ does not contain x free or quantified]
φ
====================
∀xφ


III. Interactions between ∀, ∧, ¬.

SUBSTITUTION.
∀x∀y(φ(x) ∧ Ψ(x, y))
====================
∀x(φ(x) ∧ ∀yΨ(x, y))

SUBSTITUTION.
∀x∀y¬(φ(x) ∧ Ψ(x, y))
====================
∀x¬(φ(x) ∧ ¬∀y¬Ψ(x, y))

TRANSFORMATION.
∀x¬φ(x)
\/\/\/\/\/\/\/\/\/\/
¬∀xφ(x)

TRANSFORMATION.
∀x∀yφ(x, y)
\/\/\/\/\/\/\/\/\/\/
∀xφ(x, x)


"""
pass

In [26]:
"""
Some meta-theorems (for arbitrary φ, Ψ), using compilation laws.

METATHEOREM.
φ ∨ Ψ
====================
¬(¬φ ∧ ¬Ψ)
====================
¬(¬Ψ ∧ ¬φ)
====================
Ψ ∨ φ


METATHEOREM.
φ ⇒ φ
====================
¬φ ∨ φ
====================
⊤


METATHEOREM.
φ ∧ (Ψ ∨ ξ)
====================
φ ∧ ¬(¬Ψ ∧ ¬ξ)
====================
¬(¬(φ ∧ Ψ) ∧ ¬(φ ∧ ξ))
====================
(φ ∧ Ψ) ∨ (φ ∧ ξ)


METATHEOREM.
φ ∨ (Ψ ∧ ξ)
====================
¬(¬φ ∧ ¬(Ψ ∧ ξ))
====================
¬(¬φ ∧ (¬Ψ ∨ ¬ξ))
====================
¬((¬φ ∧ ¬Ψ) ∨ (¬φ ∧ ¬ξ))
====================
(¬(¬φ ∧ ¬Ψ) ∧ ¬(¬φ ∧ ¬ξ))
====================
((φ ∨ Ψ) ∧ (φ ∨ ξ))


METATHEOREM.
φ ∧ (Ψ ∨ ξ)
====================
φ ∧ ¬(¬Ψ ∧ ¬ξ)
====================
¬(¬(φ ∧ Ψ) ∧ ¬(φ ∧ ξ))
====================
(φ ∧ Ψ) ∨ (φ ∧ ξ)

METATHEOREM.
φ ∨ (Ψ ∧ ¬Ψ)
====================
¬(¬φ ∧ ¬(Ψ ∧ ¬Ψ))
====================
¬¬φ
====================
φ


METATHEOREM.
φ ⇔ Ψ
====================
(φ ∧ Ψ) ∨ (¬φ ∧ ¬Ψ)
====================
(Ψ ∧ φ) ∨ (¬φ ∧ ¬Ψ)
====================
((¬φ ∧ φ) ∨ (Ψ ∧ φ)) ∨ ((¬φ ∧ ¬Ψ) ∨ (Ψ ∧ ¬Ψ))
====================
((¬φ ∨ Ψ) ∧ φ) ∨ ((¬φ ∨ Ψ) ∧ ¬Ψ)
====================
(¬φ ∨ Ψ) ∧ (φ ∨ ¬Ψ)
====================
(¬φ ∨ Ψ) ∧ (¬Ψ ∨ φ)
====================
(φ ⇒ Ψ) ∧ (Ψ ⇒ φ)


METATHEOREM.
φ ∧ (φ ⇒ Ψ)
====================
φ ∧ (¬φ ∨ Ψ)
====================
(φ ∧ ¬φ) ∨ (φ ∧ Ψ)
====================
φ ∧ Ψ


---


METATHEOREM.
∀x∀y(φ(x) ∨ Ψ(x, y))
====================
∀x∀y¬(¬φ(x) ∧ ¬Ψ(x, y))
====================
∀x¬(¬φ(x) ∧ ¬∀y¬¬Ψ(x, y))
====================
∀x¬(¬φ(x) ∧ ¬∀yΨ(x, y))
====================
∀x(φ(x) ∨ ∀yΨ(x, y))


METATHEOREM.
∀x∀y(φ(x) ∧ Ψ(y))
====================
∀x(φ(x) ∧ ∀yΨ(y))
====================
∀x(∀yΨ(y) ∧ φ(x))
====================
∀yΨ(y) ∧ ∀xφ(x)


METATHEOREM.
∀x∀y(φ(x) ⇒ Ψ(y))
====================
∀x∀y(¬φ(x) ∨ Ψ(y))
====================
∀x(¬φ(x) ∨ ∀yΨ(y))
====================
∀x¬φ(x) ∨ ∀yΨ(y)


METATHEOREM.
∀x∀y(φ(x) ∧ Ψ(y))
====================
∀x(φ(x) ∧ Ψ(x))


METATHEOREM.
?????
====================
∀y¬(∀xφ(x) ∧ ¬φ(y))
====================
∀y¬(∀xφ(x) ∧ ¬φ(y))
====================
∀y(¬∀xφ(x) ∨ φ(y))
====================
¬∀xφ(x) ∨ ∀yφ(y)
====================
∀xφ(x) ⇒ ∀yφ(y)


"""
pass

In [27]:
"""
METATHEOREM. Let φ be any formula (maybe with free variables). Then:

¬φ ∨ φ
====================
⊤

PROOF.
By induction on the formula length, either φ = ¬Ψ or φ = Ψ ∧ ξ.
In the first case, we have:
¬φ ∨ φ
====================
¬¬Ψ ∨ ¬Ψ
====================
Ψ ∨ ¬Ψ
====================
¬Ψ ∨ Ψ
====================
⊤
In the second case, we have
¬φ ∨ φ
====================
¬(Ψ ∧ ξ) ∨ (Ψ ∧ ξ)
====================
(¬(Ψ ∧ ξ) ∨ Ψ) ∧ (¬(Ψ ∧ ξ) ∨ ξ)
====================
((¬Ψ ∨ ¬ξ) ∨ Ψ) ∧ ((¬Ψ ∨ ¬ξ) ∨ ξ)
====================
((¬ξ ∨ ¬Ψ) ∨ Ψ) ∧ ((¬Ψ ∨ ¬ξ) ∨ ξ)
====================
(¬ξ ∨ (¬Ψ ∨ Ψ)) ∧ (¬Ψ ∨ (¬ξ ∨ ξ))
====================
(¬ξ ∨ ⊤) ∧ (¬Ψ ∨ ⊤)
====================
⊤ ∧ ⊤
====================
⊤

---------------------------

"""
pass

In [ ]:
"""
Some theorems.

THEOREM. [equal sets are those which are subsets of each other]


----------------------------------------
----------------------------------------
----------------------------------------
----------------------------------------
∀x∀y(∀c1((c1 ∈ x) ⇔ (c1 ∈ y)) ⇔ ∀c2((c2 ∈ x) ⇔ (c2 ∈ y)))
----------------------------------------
∀x∀y(∀c1((c1 ∈ x) ⇔ (c1 ∈ y)) ⇔ ∀c2(((c2 ∈ x) ⇒ (c2 ∈ y)) ∧ ((c2 ∈ y) ⇒ (c2 ∈ x))))
----------------------------------------
∀x∀y(∀c1((c1 ∈ x) ⇔ (c1 ∈ y)) ⇔ ∀c2∀c3(((c2 ∈ x) ⇒ (c2 ∈ y)) ∧ ((c3 ∈ y) ⇒ (c3 ∈ x))))
----------------------------------------
∀x∀y(∀c1((c1 ∈ x) ⇔ (c1 ∈ y)) ⇔ (∀c2((c2 ∈ x) ⇒ (c2 ∈ y)) ∧ ∀c3((c3 ∈ y) ⇒ (c3 ∈ x))))
========================================
∀x∀y((x = y) ⇔ ((x ⊂ y) ∧ (y ⊂ x)))


"""